# [ 5-5. 현금흐름 구현 ]

In [1]:
#pandas DataFrame 화면상 출력 한도 설정
import pandas as pd
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

## 1. 한단계씩 현금흐름 추적하기

In [2]:
from practice.astn0_overview import overview, idx, acc
from practice.astn1_sales import sales
from practice.astn2_cost import cost
from practice.astn3_equity import equity
from practice.astn4_loan import loan

In [3]:
idxno = idx[0]

#운영계좌 잔액 확인
_ = acc.oprtg.bal_strt[idxno]
acc.tmp.addamt(idxno, _, note="월초 잔액")

#자기자본 인출
_ = equity.withdraw_equity_amount(idxno, acc.oprtg)
acc.tmp.addamt(idxno, _, note="자기자본 납입")

#사업비 추정
oprtg_estmtd = cost.estimate_cost_amt(idxno)

lncst_estmtd = 0
for ln in loan.getloan(reverse=True):
    lncst_estmtd += ln.estimate_fee_amt(idxno)
    lncst_estmtd += ln.estimate_IR_amt(idxno)

cst_estmtd = oprtg_estmtd + lncst_estmtd
acc.tmp.subscd(idxno, cst_estmtd, note="추정 사업비")

#분양매출 인식
for sls in sales.dct.values():
    _ = sls.get_salesamt(acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"분양매출({sls.name})")

#대출원금 인출
for ln in loan.getloan(reverse=True):
    ln.set_loan_withdrawable(idxno)
    _ = ln.withdraw_ntnl_fixed(acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"일시대출금({ln.name})")
    
for ln in loan.getloan(reverse=True):
    _ = ln.withdraw_ntnl_flexible(acc.tmp, acc.oprtg, idxno)
    acc.tmp.addamt(idxno, _, note=f"한도대출금({ln.name})")
    
#금융비용 지출
for ln in loan.getloan(reverse=False):
    fee = ln.pay_fee_amt(acc.oprtg, idxno)
    IR = ln.pay_IR_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, fee, note=f"수수료({ln.name})")
    acc.tmp.subamt(idxno, IR, note=f"이자({ln.name})")

#일반사업비 지출
_ = cost.pay_cost_amt(acc.oprtg, idxno)
acc.tmp.subamt(idxno, _, note="일반사업비")

#대출원금 상환
for ln in loan.getloan(reverse=False):
    amtrpy = ln.repay_ntnl_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, amtrpy, note=f"대출금상환({ln.name})")
    ln.setback_loan_unwithdrawable(idxno)
    
#임시계좌 현금 조정
acc.tmp.subamt(idxno, acc.tmp.bal_end[idxno], note="임시계좌 잔액 조정")

In [4]:
acc.tmp.jnlscd

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,0,0.0,None,None,추정 사업비


In [5]:
acc.tmp.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정


In [6]:
acc.oprtg.df

,bal_strt,amt_in,amt_out,bal_end
2023-01-31,0.0,5000.0,0.0,5000.0
2023-02-28,5000.0,0.0,0.0,5000.0
2023-03-31,5000.0,0.0,0.0,5000.0
2023-04-30,5000.0,0.0,0.0,5000.0
2023-05-31,5000.0,0.0,0.0,5000.0
2023-06-30,5000.0,0.0,0.0,5000.0
2023-07-31,5000.0,0.0,0.0,5000.0
2023-08-31,5000.0,0.0,0.0,5000.0
2023-09-30,5000.0,0.0,0.0,5000.0
2023-10-31,5000.0,0.0,0.0,5000.0


In [7]:
acc.oprtg.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,보통주,None,자기자본 납입


## 2. For문을 통해 일괄적으로 현금흐름 추적하기

In [9]:
from practice.astn0_overview import overview, idx, acc
from practice.astn1_sales import sales
from practice.astn2_cost import cost
from practice.astn3_equity import equity
from practice.astn4_loan import loan

In [10]:
for idxno in idx:
    #운영계좌 잔액 확인
    _ = acc.oprtg.bal_strt[idxno]
    acc.tmp.addamt(idxno, _, note="월초 잔액")

    #자기자본 인출
    _ = equity.withdraw_equity_amount(idxno, acc.oprtg)
    acc.tmp.addamt(idxno, _, note="자기자본 납입")

    #사업비 추정
    oprtg_estmtd = cost.estimate_cost_amt(idxno)

    lncst_estmtd = 0
    for ln in loan.getloan(reverse=True):
        lncst_estmtd += ln.estimate_fee_amt(idxno)
        lncst_estmtd += ln.estimate_IR_amt(idxno)

    cst_estmtd = oprtg_estmtd + lncst_estmtd
    acc.tmp.subscd(idxno, cst_estmtd, note="추정 사업비")
    
    #분양매출 인식
    for sls in sales.dct.values():
        _ = sls.get_salesamt(acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"분양매출({sls.name})")

    #대출원금 인출
    for ln in loan.getloan(reverse=True):
        ln.set_loan_withdrawable(idxno)
        _ = ln.withdraw_ntnl_fixed(acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"일시대출금({ln.name})")

    for ln in loan.getloan(reverse=True):
        _ = ln.withdraw_ntnl_flexible(acc.tmp, acc.oprtg, idxno)
        acc.tmp.addamt(idxno, _, note=f"한도대출금({ln.name})")

    #금융비용 지출
    for ln in loan.getloan(reverse=False):
        fee = ln.pay_fee_amt(acc.oprtg, idxno)
        IR = ln.pay_IR_amt(acc.oprtg, idxno)
        acc.tmp.subamt(idxno, fee, note=f"수수료({ln.name})")
        acc.tmp.subamt(idxno, IR, note=f"이자({ln.name})")

    #일반사업비 지출
    _ = cost.pay_cost_amt(acc.oprtg, idxno)
    acc.tmp.subamt(idxno, _, note="일반사업비")

    #대출원금 상환
    for ln in loan.getloan(reverse=False):
        amtrpy = ln.repay_ntnl_amt(acc.oprtg, idxno)
        acc.tmp.subamt(idxno, amtrpy, note=f"대출금상환({ln.name})")
        ln.setback_loan_unwithdrawable(idxno)

    #임시계좌 현금 조정
    acc.tmp.subamt(idxno, acc.tmp.bal_end[idxno], note="임시계좌 잔액 조정")

In [11]:
acc.tmp.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정
2023-01-31,5000,0,None,None,자기자본 납입
2023-01-31,0,5000.0,None,None,임시계좌 잔액 조정
2023-02-28,10000.0,0,None,None,월초 잔액
2023-02-28,0,10000.0,None,None,임시계좌 잔액 조정
2023-03-31,10000.0,0,None,None,월초 잔액
2023-03-31,5000,0,None,None,일시대출금(trb)
2023-03-31,5000,0,None,None,일시대출금(tra)
2023-03-31,4700.0,0,None,None,한도대출금(tra)


In [12]:
acc.oprtg.jnl

,amt_in,amt_out,rcvfrm,payto,note
2023-01-31,5000,0,보통주,None,자기자본 납입
2023-01-31,5000,0,보통주,None,자기자본 납입
2023-03-31,5000,0,ntnl,None,일시대출금(trb)
2023-03-31,5000,0,ntnl,None,일시대출금(tra)
2023-03-31,4700.0,0,ntnl,None,한도대출금(tra)
...,...,...,...,...,...
2025-05-31,0,84.0,None,IR,이자(tra)
2025-05-31,0,37.5,None,IR,이자(trb)
2025-05-31,0,26.207407,None,예비비,예비비
2025-05-31,0,40000.0,None,ntnl,대출금상환(tra)


In [13]:
acc.oprtg.df

,bal_strt,amt_in,amt_out,bal_end
2023-01-31,0.000000,10000.0,0.000000,10000.000000
2023-02-28,10000.000000,0.0,0.000000,10000.000000
2023-03-31,10000.000000,14700.0,24616.407407,83.592593
2023-04-30,83.592593,2003.2,2045.164907,41.627685
2023-05-31,41.627685,600.0,587.635907,53.991778
2023-06-30,53.991778,700.0,655.530407,98.461370
2023-07-31,98.461370,500.0,550.872907,47.588463
2023-08-31,47.588463,600.0,618.267407,29.321056
2023-09-30,29.321056,1303.2,772.687907,559.833148
2023-10-31,559.833148,2606.4,1575.708407,1590.524741


In [14]:
sales.오피.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,1303.2,1303.2,0.0,0.0,0.0,1303.2,1303.2,-1303.2,0.0,0.0
2023-05-31,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0
2023-06-30,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0
2023-07-31,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0
2023-08-31,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0,0.0,1303.2,-1303.2,0.0,0.0
2023-09-30,0.0,0.0,1303.2,2606.4,-1303.2,0.0,0.0,1303.2,2606.4,-2606.4,0.0,0.0
2023-10-31,0.0,0.0,2606.4,5212.8,-2606.4,0.0,0.0,2606.4,5212.8,-5212.8,0.0,0.0


In [15]:
sales.근생.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-08-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-09-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
loan.tra.ntnl.dfall

,scd_in,scd_in_cum,scd_out,scd_out_cum,bal_strt,amt_in,amt_in_cum,amt_out,amt_out_cum,bal_end,rsdl_in_cum,rsdl_out_cum
2023-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-31,0.0,0.0,40000.0,40000.0,0.0,0.0,0.0,9700.0,9700.0,-9700.0,0.0,30300.0
2023-04-30,0.0,0.0,0.0,40000.0,-9700.0,0.0,0.0,700.0,10400.0,-10400.0,0.0,29600.0
2023-05-31,0.0,0.0,0.0,40000.0,-10400.0,0.0,0.0,600.0,11000.0,-11000.0,0.0,29000.0
2023-06-30,0.0,0.0,0.0,40000.0,-11000.0,0.0,0.0,700.0,11700.0,-11700.0,0.0,28300.0
2023-07-31,0.0,0.0,0.0,40000.0,-11700.0,0.0,0.0,500.0,12200.0,-12200.0,0.0,27800.0
2023-08-31,0.0,0.0,0.0,40000.0,-12200.0,0.0,0.0,600.0,12800.0,-12800.0,0.0,27200.0
2023-09-30,0.0,0.0,0.0,40000.0,-12800.0,0.0,0.0,0.0,12800.0,-12800.0,0.0,27200.0
2023-10-31,0.0,0.0,0.0,40000.0,-12800.0,0.0,0.0,0.0,12800.0,-12800.0,0.0,27200.0
